Problem Statement

Given a data stream input of non-negative integers a1, a2, ..., an, summarize the numbers seen so far as a list of disjoint intervals.

Implement the SummaryRanges class:

    SummaryRanges() Initializes the object with an empty stream.
    void addNum(int val) Adds the integer val to the stream.
    int[][] getIntervals() Returns a summary of the integers in the stream currently as a list of disjoint intervals [starti, endi].

 

Example 1:

Input
["SummaryRanges", "addNum", "getIntervals", "addNum", "getIntervals", "addNum", "getIntervals", "addNum", "getIntervals", "addNum", "getIntervals"]
[[], [1], [], [3], [], [7], [], [2], [], [6], []]
Output
[null, null, [[1, 1]], null, [[1, 1], [3, 3]], null, [[1, 1], [3, 3], [7, 7]], null, [[1, 3], [7, 7]], null, [[1, 3], [6, 7]]]

Explanation
SummaryRanges summaryRanges = new SummaryRanges();
summaryRanges.addNum(1);      // arr = [1]
summaryRanges.getIntervals(); // return [[1, 1]]
summaryRanges.addNum(3);      // arr = [1, 3]
summaryRanges.getIntervals(); // return [[1, 1], [3, 3]]
summaryRanges.addNum(7);      // arr = [1, 3, 7]
summaryRanges.getIntervals(); // return [[1, 1], [3, 3], [7, 7]]
summaryRanges.addNum(2);      // arr = [1, 2, 3, 7]
summaryRanges.getIntervals(); // return [[1, 3], [7, 7]]
summaryRanges.addNum(6);      // arr = [1, 2, 3, 6, 7]
summaryRanges.getIntervals(); // return [[1, 3], [6, 7]]

 

Constraints:

    0 <= val <= 104
    At most 3 * 104 calls will be made to addNum and getIntervals.

 

Follow up: What if there are lots of merges and the number of disjoint intervals is small compared to the size of the data stream?

# Min Heap - O(Log N) addNum, O(N * Log N) getIntervals

In [1]:
from typing import List
from heapq import heappush, heappop

class SummaryRanges:

    def __init__(self):
        self.intervals = []
        self.seen = set()

    def addNum(self, val: int) -> None:
        if val not in self.seen:
            self.seen.add(val)
            heappush(self.intervals, [val, val])

    def getIntervals(self) -> List[List[int]]:
        tmp = []

        while self.intervals:
            cur = heappop(self.intervals)
            if tmp and cur[0] <= tmp[-1][1] + 1:
                tmp[-1][1] = max(tmp[-1][1], cur[1])
            else:
                tmp.append(cur)

        self.intervals = tmp
        
        return self.intervals

# Union Find - O(1) for addNum, O(N * log N) for getIntevals where N is number of unique intervals

In [3]:
from typing import List

class DSU:
    def __init__(self):
        self.p = {}
        self.ranks = {}
        self.intervals = {}

    def exists(self, x): return x in self.p

    def make_set(self, x):
        self.p[x] = x
        self.ranks[x] = 1
        self.intervals[x] = [x,x]
        
    def find(self, x):
        if not self.exists(x): return None
        
        if x != self.p[x]:
            self.p[x] = self.find(self.p[x])

        return self.p[x]

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)
        
        if x is None or y is None or x == y: return
        if self.ranks[x] < self.ranks[y]:
            x, y = y, x
        self.ranks[x] += self.ranks[x] == self.ranks[y]
        
        self.p[y] = x
        
        ## interval adjusting logic
        self.intervals[x] = [min(self.intervals[x][0], self.intervals[y][0]), max(self.intervals[x][1], self.intervals[y][1])]

        self.intervals.pop(y)
        
        
class SummaryRanges:

    def __init__(self):
        self.dsu = DSU()

    def addNum(self, val: int) -> None:
        if self.dsu.exists(val): return
            
        self.dsu.make_set(val)
        
        self.dsu.union(val, val-1)
        self.dsu.union(val, val+1)

    def getIntervals(self) -> List[List[int]]:
        return sorted(self.dsu.intervals.values())
